In [ ]:
from __future__ import print_function, division

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(context="poster")
import ipywidgets
import yt
import glob
import os
import warnings
import h5py

import numpy as np
import pandas as pd

from units import M_solar, m_proton, pc, yr, Myr, km, s, gamma

from injection_helpers import get_SNe

from visualize_helpers import \
    get_snapshot_filenames, \
    total_mass_of_snapshot, \
    total_radial_momentum_of_snapshot, \
    map_to_all_snapshots, \
    get_snapshot_times, \
    total_kinetic_energy_of_snapshot, \
    total_internal_energy_of_snapshot, \
    load_snapshots
    

@yt.derived_field(name="pressure", units="g  / s**2 / cm")
def _pressure(field, data):
    return (gamma-1) * data["thermal_energy"] * data["density"]

In [ ]:
# run_dir = "../runs/cluster/"
# run_dir = "../runs/cluster_cooling/"
# run_dir = "../runs/cluster_cooling_150/"
# run_dir = "../runs/cluster_cooling_200/"
# run_dir = "../runs/cluster_cooling_250/"
run_dir = "../runs/cluster_cooling_300/"
# run_dir = "../runs/single/"
# run_dir = "../runs/single_cooling/"
# run_dir = "../runs/double/"
# run_dir = "../runs/double_cooling/"

inputs_dir = os.path.join(run_dir, "inputs")
outputs_dir = os.path.join(run_dir, "outputs")

# Warning
This file is very much a work-in-progress.

To do:
 - implement a shock-finder
 - only get energy within the remnant

# Overview

In [ ]:
SNe = get_SNe(inputs_dir)
SN_times           = np.array([SN.time          for SN in SNe])
SN_ejecta_masses   = np.array([SN.ejecta_mass   for SN in SNe])
SN_ejecta_masses_Z = np.array([SN.ejecta_mass_Z for SN in SNe])

In [ ]:
print(SN_times)

In [ ]:
print(SN_ejecta_masses)

In [ ]:
ts = load_snapshots(outputs_dir)
snapshot_filenames = get_snapshot_filenames(outputs_dir)
times_snapshots = get_snapshot_times(ts)

print("Loaded {} snapshots".format(len(ts)))

ds = ts[0]
rho_0 = ds.all_data()["all","density"].mean()


In [ ]:
times_snapshots

In [ ]:
print( "Length unit: ",   ds.length_unit)
print( "Time unit: ",     ds.time_unit)
print( "Mass unit: ",     ds.mass_unit)
print( "Velocity unit: ", ds.velocity_unit)

# Plot Global Quantities

In [ ]:
ds = ts[0]
dd = ds.all_data()

In [ ]:
ds.field_list

In [ ]:
energies = np.loadtxt(os.path.join(outputs_dir, "energy.txt"), ndmin=2)

times_statistics    = energies[:,0] * ds.time_unit
thermal_energies    = energies[:,1] * ds.mass_unit * (ds.velocity_unit)**2
potential_energies  = energies[:,2] * ds.mass_unit * (ds.velocity_unit)**2
kinetic_energies    = energies[:,3] * ds.mass_unit * (ds.velocity_unit)**2

times_statistics    = times_statistics.convert_to_cgs().value / Myr
thermal_energies    = thermal_energies.convert_to_cgs().value
potential_energies  = potential_energies.convert_to_cgs().value
kinetic_energies    = kinetic_energies.convert_to_cgs().value


total_mass    = energies[:,-6]


total_energies = thermal_energies + kinetic_energies

sorted_times_statistics_indices = np.argsort(times_statistics)

In [ ]:
kinetic_energies

In [ ]:
_e_kin = map_to_all_snapshots(outputs_dir, total_kinetic_energy_of_snapshot)
_e_int = map_to_all_snapshots(outputs_dir, total_internal_energy_of_snapshot)
_e_tot = _e_kin + _e_int

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_statistics[sorted_times_statistics_indices],
         kinetic_energies[sorted_times_statistics_indices],
         label="statistics")
plt.plot(times_snapshots, _e_kin, label="snapshots")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$E_\mathrm{kin}$ $[\mathrm{ergs}]$")
plt.legend(loc="best")

In [ ]:
thermal_energies

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_statistics[sorted_times_statistics_indices],
         thermal_energies[sorted_times_statistics_indices],
         label="statistics",
        )
plt.plot(times_snapshots, _e_int, label="snapshots")
plt.legend(loc="best")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$E_\mathrm{int}$ $[\mathrm{ergs}]$")

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_statistics[sorted_times_statistics_indices],
         total_energies[sorted_times_statistics_indices],
         label="statistics")
plt.plot(times_snapshots, _e_int, label="snapshots")
plt.legend(loc="best")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$E_\mathrm{total}$ $[\mathrm{ergs}]$")

To do: remove the contribution from cooling outside the remnant

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_statistics[sorted_times_statistics_indices], 
         total_energies[sorted_times_statistics_indices] - total_energies[sorted_times_statistics_indices][0],
         label="statistics")
plt.plot(times_snapshots, _e_tot - _e_tot[0], label="snapshots")
plt.legend(loc="best")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$\Delta E_\mathrm{total}$ $[\mathrm{ergs}]$")

In [ ]:
print((total_energies - total_energies[0]) / 1e51)

# Plot Timing
Can be useful for noticing if the energy file doubles back on itself

In [ ]:
plt.plot(times_snapshots)

In [ ]:
plt.plot(times_statistics)

## Mass Plots

WARNING: in "energy.txt" GIZMO only uses %g formatting; the change in total mass due to ejecta might be truncated 

In [ ]:
masses = map_to_all_snapshots(outputs_dir, total_mass_of_snapshot)

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_snapshots, masses - masses[0], 
         label="snapshots", linestyle="solid", drawstyle="steps-post")
plt.plot(SN_times, SN_ejecta_masses.cumsum(), 
         label="intended", linestyle="dashed", drawstyle="steps-post")
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$\Delta M$ $[M_\odot]$")
plt.legend(loc="best")

In [ ]:
times_snapshots[-3:]

In [ ]:
SN_times[4]

## Momentum Plots

In [ ]:
radial_momentum = map_to_all_snapshots(outputs_dir, total_radial_momentum_of_snapshot)

In [ ]:
radial_momentum

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_snapshots, radial_momentum / (100 * M_solar * km / s))

plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$p$ $[100$ $M_\odot$ $\mathrm{km}$ $\mathrm{s}^{-1}]$")
plt.ylim(ymin=0)

In [ ]:
sns.rugplot(SN_times, color="k", linewidth=3)
plt.plot(times_snapshots, radial_momentum / (100 * M_solar * km / s * SN_times.size))
plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
plt.ylabel(r"$p$ $[100$ $M_\odot$ $N_\mathrm{SNe}$ $\mathrm{km}$ $\mathrm{s}^{-1}]$")
plt.ylim(ymin=0)

# Plot Snapshot Views

In [ ]:
def show_projected_density(i):
    ds = ts[i]
    
    p = yt.ProjectionPlot(ds, "x", ("gas","density"))
    p.set_cmap(field="density", cmap="viridis")
    p.annotate_timestamp(corner="upper_left", draw_inset_box=True)
    
    t = ds.current_time.convert_to_cgs().value / Myr
    N_SNe_so_far = np.sum(t > SN_times)
    p.annotate_text((.8,.94), 
                    "N_SNe: {}".format(N_SNe_so_far),
                    coord_system="axis",
                    inset_box_args={"facecolor":"darkslategray",
                                       "alpha":0.9},
                   )
    p.show()
    
ipywidgets.interact(show_projected_density,
                    i=ipywidgets.IntSlider(min=0,
                                           max=len(ts)-1,
                                           value=len(ts)-1))

In [ ]:
field_type = {
    "density": "gas",
    "temperature": "gas",
    "pressure": "gas",
    "velocity_magnitude": "gas",
    "radius": "index",
    "metallicity": "gas"
}

def show_sliced_field(i, field):
    ds = ts[i]
    s = yt.SlicePlot(ds, "z", (field_type[field], field))
    s.set_cmap(field=field, cmap="viridis")
    s.annotate_timestamp(corner="upper_left", draw_inset_box=True)
    t = ds.current_time.convert_to_cgs().value / Myr
    N_SNe_so_far = np.sum(t > SN_times)
    s.annotate_text((.8,.94), 
                    "N_SNe: {}".format(N_SNe_so_far),
                    coord_system="axis",
                    inset_box_args={"facecolor":"darkslategray",
                                       "alpha":0.9},
                   )
    s.show()
    
ipywidgets.interact(show_sliced_field,
                i=ipywidgets.IntSlider(min=0,
                                       max=len(ts)-1,
                                       value=len(ts)-1),
                field = ipywidgets.Dropdown(options=list(field_type.keys()),
                                            value="density"))

# Profiles

In [ ]:
def create_density_profile(ds, n_bins=20):
    dd = ds.all_data()
    r_max = ds.domain_width[0]/2

    dr = r_max / n_bins

    rs = np.linspace(0, r_max.value, num=n_bins+1)[1:]

    dmass = np.zeros(n_bins)
    ones = np.zeros(n_bins, dtype=int)

    for i in range(n_bins):
        r_i = dr*(i)
        r_o = dr*(i+1)

        mask =    (dd["all", "particle_position_spherical_radius"] >= r_i) \
                & (dd["all", "particle_position_spherical_radius"] <  r_o)

#         ones[i] = mask.sum()
        dmass[i] = dd["all", "Masses"][mask].sum().convert_to_cgs().value
            
    Vs = 4/3*np.pi*rs**3
    Vs = np.insert(Vs, 0, 0)
    dVs = Vs[1:] - Vs[:-1]

    densities = dmass / (dVs * pc**3)
    
    return rs, densities

In [ ]:
field_y_labels = {
    "density" : r"$\rho$ $[\mathrm{m_p}$ $\mathrm{cm}^{-3}]$",
    "temperature" : r"$T$ $[\mathrm{K}]$",
    "pressure" : r"$P$ $[\mathrm{ergs}$ $\mathrm{cm}^{-3}]$",
    "velocity_magnitude" : r"$\|\mathbf{v}\|$ $[\mathrm{km}$ $\mathrm{s}^{-1}]$",
    "radial_velocity" : r"$v_r$ $[\mathrm{km}$ $\mathrm{s}^{-1}]$",
    "Metallicity" : r"$Z / Z_\odot$",
}

field_weight = {
    "temperature" : "cell_mass",
    "pressure" : "cell_volume",
    "velocity_magnitude" : "cell_mass",
    "radial_velocity" : "cell_mass",
    "Metallicity" : "cell_mass",
}

field_units = {
    "density" : m_proton,
    "temperature" : 1,
    "pressure" : 1,
    "velocity_magnitude" : km / s, 
    "radial_velocity"    : km / s, 
    "Metallicity" : 0.02,
}



def show_profile(i, field):
    ds = ts[i]
    sp = ds.sphere(ds.domain_center, ds.domain_width[0]/2)
    
    
    if field is "density":
        rs, densities = create_density_profile(ds,n_bins=64)
        plt.plot(rs, densities / field_units[field])

        plt.ylim(ymin=1e-4)
        
        plt.axhline(rho_0, linestyle="dashed", color="k")
        
    else:    
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            pp = yt.create_profile(sp, 
                                   "radius", [field, "ones"], 
                                   weight_field=field_weight[field],
                                   units = {"radius":"pc"},
                                   logs = {"radius":False},
                                   n_bins=64,
            )
        mask = pp["ones"] > 0.1 # filter out bins with no particles
        plt.plot(pp.x.value[mask], 
                 pp[field][mask] / field_units[field])

    plt.yscale("log")
    plt.ylabel(field_y_labels[field])

    plt.xlabel(r"$R$ $[\mathrm{pc}]$")
    
    if times_snapshots[i] < 0:
        raise RuntimeError("Invalid time: {}".format(time_snapshots[i]))
    elif times_snapshots[i] < 1e-3:
        time = times_snapshots[i]
        time_units = "Myr"
        title = r"$t$ $= {:.1e}$ $\mathrm{{{}}}$".format(time, time_units)
    elif times_snapshots[i] < 1:
        time = times_snapshots[i] * 1e3
        time_units = "kyr"
        title = r"$t$ $= {:.0f}$ $\mathrm{{{}}}$".format(time, time_units)
    elif times_snapshots[i] < 10:
        time = times_snapshots[i]
        time_units = "Myr"
        title = r"$t$ $= {:.1f}$ $\mathrm{{{}}}$".format(time, time_units)
    else:
        time = times_snapshots[i]
        time_units = "Myr"
        title = r"$t$ $= {:.0f}$ $\mathrm{{{}}}$".format(time, time_units)
        
    plt.title(title)
    

ipywidgets.interact(show_profile,
                i=ipywidgets.IntSlider(min=0,
                                       max=len(ts)-1,
                                       value=len(ts)-1),
                field = ipywidgets.Dropdown(options=list(field_y_labels),
                                            value="density"))